In [1]:
import numpy as np
import pandas as pd
import re

In [2]:
cars = pd.read_csv('sg_used_cars.csv',thousands=',')

In [3]:
cars.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   name      1000 non-null   object 
 1   price     1000 non-null   object 
 2   depre     970 non-null    float64
 3   mileage   870 non-null    float64
 4   eng_cap   993 non-null    float64
 5   reg_date  996 non-null    object 
 6   power     981 non-null    float64
 7   owners    1000 non-null   object 
dtypes: float64(4), object(4)
memory usage: 62.6+ KB


# 1) Create new columns #

### Split the 'name' column into 'brand' and 'model' ###

In [4]:
cars['brand'] = cars['name'].apply(lambda x:x.split()[0])

In [5]:
cars.brand.unique()

array(['Toyota', 'Audi', 'Mazda', 'Lexus', 'Mercedes-Benz', 'BMW',
       'Volkswagen', 'Jaguar', 'Volvo', 'Maserati', 'Honda', 'Hyundai',
       'Tesla', 'Bentley', 'Rolls-Royce', 'Infiniti', 'Nissan', 'Kia',
       'Subaru', 'Opel', 'Skoda', 'Peugeot'], dtype=object)

In [6]:
def car_model(x):
    model = ' '.join(x.split()[1:])
    return model

cars['model'] = np.vectorize(car_model)(cars['name'])

In [7]:
cars.drop('name',axis=1,inplace=True)

In [8]:
front_two_columns = ['brand','model']
cars = cars[front_two_columns+[col for col in cars if col not in front_two_columns]]

### Clean up the owners column by removing any leading and trailing whitespaces

In [9]:
cars.rename(columns = {'owners ':'owners'},inplace = True)

In [10]:
cars['owners'] = np.vectorize(lambda x:x.strip())(cars['owners'])

### Create a column to mark whether a car has its COE extended

In [11]:
def is_coe_extended(model):
    if re.findall(r'\(.*COE.*\)',model):
        return True
    else:
        return False

cars['COE_extended'] = np.vectorize(is_coe_extended)(cars['model'])

### Create a column for the maturity date of the car

#### Find maturity date based on:
1) if its COE is extended and the new date is shown

2) if its COE is newly extended, take it that maturity date is 20 years from registration date

3) if its first COE has not expired, maturity date will be 10 years from registration date 



In [12]:
cars['reg_date'] = pd.to_datetime(cars['reg_date'])

In [13]:
def maturity_date(coe_extended,model,reg_date):
    if coe_extended:
        if re.findall(r'\d{2}/\d{4}',model):
            maturity_date = re.findall(r'\d{2}/\d{4}',model)[0]
        else:
            maturity_date = (pd.to_datetime(reg_date) + np.timedelta64(20,'Y')).date()
    else:
        maturity_date = (pd.to_datetime(reg_date) + np.timedelta64(10,'Y')).date()
    
    return maturity_date

cars['maturity_date'] = np.vectorize(maturity_date)(cars['COE_extended'],cars['model'],cars['reg_date'])

In [14]:
# remove coe date from car model

def remove_coe(model):
    if re.findall(r'\(.*COE.*\)',model):
        coe = re.findall(r'\(.*COE.*\)',model)[0]
        model = model.replace(coe,'')
    return model


cars['model'] = np.vectorize(remove_coe)(cars['model'])

In [15]:
cars['maturity_date'] = pd.to_datetime(cars['maturity_date'])

In [16]:
cars

,brand,model,price,depre,mileage,eng_cap,reg_date,power,owners,COE_extended,maturity_date
0,Toyota,Camry 2.4A,"59,800",12320.0,110348.0,2362.0,2007-11-19,123.0,3,True,2027-11-01
1,Toyota,Camry 2.0A,"59,800",9740.0,141813.0,1998.0,2009-03-02,108.0,5,True,2029-02-01
2,Toyota,Camry 2.4A,"55,800",9380.0,NaN,2362.0,2008-12-22,123.0,2,True,2028-12-01
3,Audi,A3 Sedan Mild Hybrid 1.5A TFSI S-tronic,"151,500",16610.0,10000.0,1498.0,2021-08-20,110.0,1,False,2031-08-20
4,Mazda,6 2.0A Executive,"73,800",14380.0,74000.0,1998.0,2017-04-28,121.0,1,False,2027-04-28
...,...,...,...,...,...,...,...,...,...,...,...
995,Nissan,Teana 2.5A,"63,800",9370.0,150000.0,2496.0,2009-11-10,134.0,1,True,2029-10-01
996,Mercedes-Benz,CLA-Class CLA180 AMG Line,"130,800",18240.0,46000.0,1595.0,2019-05-09,90.0,2,False,2029-05-08
997,Audi,S8 4.0A TFSI Quattro,"239,800",23960.0,112000.0,3993.0,2013-02-28,382.0,5,True,2033-02-27
998,Toyota,Camry Hybrid 2.5A Ascent,"115,800",16740.0,50627.0,2487.0,2019-05-03,160.0,2,False,2029-05-02


### Create a new column for the number of years left in the car ###

Find the time difference between the current reference date (2023-01-11) and the maturity date

In [17]:
def years_left(coe_extended,reg_date,maturity_date):
    
    years_left = (pd.to_datetime(maturity_date) - pd.to_datetime('2023-01-11')) / np.timedelta64(1,'Y')
    
    return round(years_left,1)

cars['years_left'] = np.vectorize(years_left)(cars['COE_extended'],cars['reg_date'],cars['maturity_date'])

### Create a new column for the age of the car ###

Find the time difference between the current reference date (2023-01-11) and the registration date

In [18]:
cars['age'] = round((pd.to_datetime('2023-01-11')-cars['reg_date'])/np.timedelta64(1,'Y'),1)

### Re-arrange order of column names ###

In [19]:
cars = cars[['brand','model','price','depre','age','mileage','eng_cap','power','owners','reg_date','maturity_date','years_left','COE_extended']]

# 2. Check for null values #

In [20]:
cars.isnull().sum()

brand              0
model              0
price              0
depre             30
age                4
mileage          130
eng_cap            7
power             19
owners             0
reg_date           4
maturity_date      4
years_left         4
COE_extended       0
dtype: int64

### Drop rows with no price & convert data type to numeric ###

In [21]:
cars = cars[cars['price'] != '.A']
cars['price'] = pd.to_numeric(cars['price'].apply(lambda x:x.replace(',','')))

### Drop rows with no depreciation or registration date ###

In [22]:
cars.dropna(how='any',subset=['depre','reg_date'],inplace=True)

In [23]:
cars.isnull().sum()

brand              0
model              0
price              0
depre              0
age                0
mileage          121
eng_cap            7
power             14
owners             0
reg_date           0
maturity_date      0
years_left         0
COE_extended       0
dtype: int64

# 3. Check for any duplicated rows #

In [24]:
cars.duplicated().sum()

7

In [25]:
cars[cars.duplicated(keep=False)]

,brand,model,price,depre,age,mileage,eng_cap,power,owners,reg_date,maturity_date,years_left,COE_extended
99,Mercedes-Benz,E-Class E250 CGI Sunroof,101800,13350.0,12.4,128000.0,1796.0,150.0,1,2010-08-27,2030-08-01,7.6,True
100,Mercedes-Benz,E-Class E250 CGI Sunroof,101800,13350.0,12.4,128000.0,1796.0,150.0,1,2010-08-27,2030-08-01,7.6,True
199,Mazda,6 2.0A,89800,13470.0,4.0,95840.0,1998.0,121.0,1,2019-01-02,2029-01-01,6.0,False
200,Mazda,6 2.0A,89800,13470.0,4.0,95840.0,1998.0,121.0,1,2019-01-02,2029-01-01,6.0,False
399,Mercedes-Benz,CLA-Class CLA180 AMG Line,165800,18490.0,2.2,28000.0,1332.0,100.0,1,2020-10-24,2030-10-24,7.8,False
400,Mercedes-Benz,CLA-Class CLA180 AMG Line,165800,18490.0,2.2,28000.0,1332.0,100.0,1,2020-10-24,2030-10-24,7.8,False
797,Toyota,Camry 2.5A,119800,14770.0,3.2,25500.0,2487.0,154.0,1,2019-10-25,2029-10-24,6.8,False
798,Toyota,Camry 2.0A,69888,15680.0,6.6,97000.0,1998.0,123.0,2,2016-06-17,2026-06-17,3.4,False
799,BMW,2 Series 216i Gran Coupe Sport,158999,15460.0,0.5,6000.0,1499.0,80.0,1,2022-07-13,2032-07-12,9.5,False
800,Toyota,Camry 2.5A,119800,14770.0,3.2,25500.0,2487.0,154.0,1,2019-10-25,2029-10-24,6.8,False


### Drop all duplicated rows ###

In [26]:
cars.drop_duplicates()

,brand,model,price,depre,age,mileage,eng_cap,power,owners,reg_date,maturity_date,years_left,COE_extended
0,Toyota,Camry 2.4A,59800,12320.0,15.1,110348.0,2362.0,123.0,3,2007-11-19,2027-11-01,4.8,True
1,Toyota,Camry 2.0A,59800,9740.0,13.9,141813.0,1998.0,108.0,5,2009-03-02,2029-02-01,6.1,True
2,Toyota,Camry 2.4A,55800,9380.0,14.1,NaN,2362.0,123.0,2,2008-12-22,2028-12-01,5.9,True
3,Audi,A3 Sedan Mild Hybrid 1.5A TFSI S-tronic,151500,16610.0,1.4,10000.0,1498.0,110.0,1,2021-08-20,2031-08-20,8.6,False
4,Mazda,6 2.0A Executive,73800,14380.0,5.7,74000.0,1998.0,121.0,1,2017-04-28,2027-04-28,4.3,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,Nissan,Teana 2.5A,63800,9370.0,13.2,150000.0,2496.0,134.0,1,2009-11-10,2029-10-01,6.7,True
996,Mercedes-Benz,CLA-Class CLA180 AMG Line,130800,18240.0,3.7,46000.0,1595.0,90.0,2,2019-05-09,2029-05-08,6.3,False
997,Audi,S8 4.0A TFSI Quattro,239800,23960.0,9.9,112000.0,3993.0,382.0,5,2013-02-28,2033-02-27,10.1,True
998,Toyota,Camry Hybrid 2.5A Ascent,115800,16740.0,3.7,50627.0,2487.0,160.0,2,2019-05-03,2029-05-02,6.3,False


## 4. Clean dataframe

Total 970 rows with 12 columns

In [27]:
cars

,brand,model,price,depre,age,mileage,eng_cap,power,owners,reg_date,maturity_date,years_left,COE_extended
0,Toyota,Camry 2.4A,59800,12320.0,15.1,110348.0,2362.0,123.0,3,2007-11-19,2027-11-01,4.8,True
1,Toyota,Camry 2.0A,59800,9740.0,13.9,141813.0,1998.0,108.0,5,2009-03-02,2029-02-01,6.1,True
2,Toyota,Camry 2.4A,55800,9380.0,14.1,NaN,2362.0,123.0,2,2008-12-22,2028-12-01,5.9,True
3,Audi,A3 Sedan Mild Hybrid 1.5A TFSI S-tronic,151500,16610.0,1.4,10000.0,1498.0,110.0,1,2021-08-20,2031-08-20,8.6,False
4,Mazda,6 2.0A Executive,73800,14380.0,5.7,74000.0,1998.0,121.0,1,2017-04-28,2027-04-28,4.3,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,Nissan,Teana 2.5A,63800,9370.0,13.2,150000.0,2496.0,134.0,1,2009-11-10,2029-10-01,6.7,True
996,Mercedes-Benz,CLA-Class CLA180 AMG Line,130800,18240.0,3.7,46000.0,1595.0,90.0,2,2019-05-09,2029-05-08,6.3,False
997,Audi,S8 4.0A TFSI Quattro,239800,23960.0,9.9,112000.0,3993.0,382.0,5,2013-02-28,2033-02-27,10.1,True
998,Toyota,Camry Hybrid 2.5A Ascent,115800,16740.0,3.7,50627.0,2487.0,160.0,2,2019-05-03,2029-05-02,6.3,False


In [28]:
cars.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 970 entries, 0 to 999
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   brand          970 non-null    object        
 1   model          970 non-null    object        
 2   price          970 non-null    int64         
 3   depre          970 non-null    float64       
 4   age            970 non-null    float64       
 5   mileage        849 non-null    float64       
 6   eng_cap        963 non-null    float64       
 7   power          956 non-null    float64       
 8   owners         970 non-null    object        
 9   reg_date       970 non-null    datetime64[ns]
 10  maturity_date  970 non-null    datetime64[ns]
 11  years_left     970 non-null    float64       
 12  COE_extended   970 non-null    bool          
dtypes: bool(1), datetime64[ns](2), float64(6), int64(1), object(3)
memory usage: 99.5+ KB


# 5. Save the dataframe as CSV #

In [29]:
cars.to_csv('cleaned_cars.csv',index=False)